In [3]:

import pandas as pd
import numpy as np


from sqlalchemy import create_engine


ck_properties = {  # clickhouse 数据库配置
    # "driver": "com.github.housepower.jdbc.ClickHouseDriver",
    "driver": "ru.yandex.clickhouse.ClickHouseDriver",
    "socket_timeout": "300000",
    "rewriteBatchedStatements": "true",
    "batchsize": "1000000",
    "numPartitions": "8",
    "user": "default",
    "password": "5NsWWQeJ",
    "host": "10.129.165.72",
    "database": 'sdecdmp',
    "port": 8123
}


mysql_properties_prod = {  # mysql 生产环境配置
    "host": "10.129.41.147",
    "port": "4918",
    "url": "jdbc:mysql://10.129.41.147:4918/ck",
    "user": "dmp_sys_guest",
    "password": "fTe^3hy*dO_363",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

mysql_properties_dev = {  # mysql 开发环境配置
    "host": "10.179.98.13",
    "port": "3306",
    "url": "jdbc:mysql://10.179.98.13:3306/ck",
    "user": "root",
    "password": "DMP_test135#",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

ck_engine = create_engine(
    f"clickhouse://{ck_properties['user']}:{ck_properties['password']}@{ck_properties['host']}:{ck_properties['port']}/{ck_properties['database']}"
)

mysql_engine_prod = create_engine(
    f"mysql+pymysql://{mysql_properties_prod['user']}:{mysql_properties_prod['password']}" +
    f"@{mysql_properties_prod['host']}:{mysql_properties_prod['port']}/{mysql_properties_prod['database']}?charset=utf8"
)

mysql_engine_dev = create_engine(
    f"mysql+pymysql://{mysql_properties_dev['user']}:{mysql_properties_dev['password']}" +
    f"@{mysql_properties_dev['host']}:{mysql_properties_dev['port']}/{mysql_properties_dev['database']}?charset=utf8"
)
                 

In [4]:
# 读取原始数据ein与uploadTime
date = '2023-07-08'
org_df = pd.read_sql(
    sql=f"""
    select
        ein, toUnixTimestamp(uploadTime1) as clt_timestamp
    from
        sdecdmp.SDECData2M_all
    where
        uploadDate = '{date}' and ein <> ''
    """,
    con=ck_engine
)
org_df.shape

(77236094, 2)

In [5]:
etl_df = pd.read_sql(
    sql=f"""
    select
        ein, clt_timestamp
    from
        sdecdmp.etl_data_v2
    where
        clt_date = '{date}'
    """,
    con=ck_engine
)
etl_df.shape

(77236094, 2)

In [35]:
org_df = org_df.loc[org_df['ein']!='']
org_df.loc[:, 'ein'] = org_df['ein'].map(lambda x: x[1:] if x[0] == '0' else x)
org_df.loc[:, 'mark'] = [1] * org_df.shape[0]
org_df.head()

,ein,clt_timestap,mark
0,D922C010451,1688782661,1
1,D922C010451,1688782662,1
2,D922C010451,1688782663,1
3,D922C010451,1688782664,1
4,D922C010451,1688782665,1


In [31]:
etl_df.head()

,ein,clt_timestamp
0,K9208000200,1688767716
1,K9208000200,1688767717
2,K9208000200,1688767718
3,K9208000200,1688767719
4,K9208000200,1688767720
